<img src="https://camo.githubusercontent.com/5535944a613e60c9be4d3a96e3d9bd34e5aba5cddc1aa6c6153123a958698289/68747470733a2f2f646f63732e64656c74612e696f2f6c61746573742f5f7374617469632f64656c74612d6c616b652d77686974652e706e67" width="150">

---
<h2 align="center">Dalta Lake Documentation</h2>
<h5 align="right">Auther: Pasit Y.</h5>

---
<h3 style="color: red;">
    ** Hive ไม่สามารถอ่าน Delta Snapshot ได้ แต่สามารถอ่าน ไฟล์ parquet โดยไม่สนใจ checkpoint โดยการสร้าง schema ดังนี้
</h3>

```sql
CREATE EXTERNAL TABLE `delta_lake.example_table` (
  `id` string, 
  `name` string
)
STORED AS PARQUET
LOCATION '/path/to/delta/table';
```

# Delta PySpark
<b>Link สำหรับการติดตั้งและ Document</b>

```https://docs.delta.io/```

```https://blog.devgenius.io/pyspark-setup-delta-lake-971e2e37330d```

```https://kontext.tech/article/1175/introduction-to-delta-lake-with-pyspark```

---
# Pyspark Config (เพิ่มจากเดิมที่มีอยู่แล้ว)

```bash
conf = pyspark.SparkConf().setAll([
     ('spark.jars.packages', 'io.delta:delta-core_2.12:2.2.0'),
     ('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension'),
     ('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog'),
     ('spark.sql.warehouse.dir', '/user/hive/warehouse/delta_lake/'),
     ('spark.databricks.delta.replaceWhere.constraintCheck.enabled', 'false'),
     ('spark.databricks.delta.replaceWhere.dataColumns.enabled', 'true'),
     ('spark.databricks.delta.schema.autoMerge.enabled','true'),
    ])
```
---
# TrinoDB Catalog
```bash
connector.name=delta-lake
hive.metastore.uri=thrift://192.168.10.40:9083
delta.compression-codec=SNAPPY
delta.max-partitions-per-writer=100
delta.enable-non-concurrent-writes=true
```
### Register existing table
```sql
CALL delta_lake.system.register_table(
    	schema_name => 'delta_lake',
    	table_name => 'tableName',
    	table_location => 'hdfs://HDFS/user/hive/warehouse/delta_lake/tableName'
)
```
### Optimize VACUUM
```sql
CALL delta_lake.system.vacuum('delta_lake', 'tableName', '7d');
```
---
# PrestoDB Catalog
```bash
connector.name=delta
hive.metastore.uri=thrift://192.168.10.40:9083
```
### การสร้าง Table External เพื่อ Link ไปยัง Delta Lake Existing
```sql
CREATE TABLE
	delta_lake.delta_lake.tableName (
        col_name INT
)
WITH (external_location = 'hdfs://HDFS/user/hive/warehouse/delta_lake/tableName');
```
### ฟังค์ชั่นนี้สามารถใช้งานได้แค่ PrestoDB เท่านั้น (Query Directory Path)
```sql
SELECT
	*
FROM
	delta_lake."$path$"."hdfs://HDFS/user/hive/warehouse/delta_lake/tableName"
LIMIT 200;
--Query แบบเลือก Snapshot version
SELECT * FROM delta_lake.delta_lake."tableName@v4" LIMIT 200;
```
---
# Config /etc/hosts
<b>ใส่ Nameservice ของ hadoop ที่เป็น Active namenode สามารถเข้าไปดูได้ที่ core-site.xml</b><br>
<b style="color: red;">** ตรวจสอบให้ถูกต้องว่า node ไหน active จะมีผลกับการเข้าไปอ่านใน metastore</b>

<b style="color: red;">HDFSSOFTNIX</b>

```py
192.168.10.40   nn01.bigdata HDFSSOFTNIX
192.168.10.41   nn02.bigdata
192.168.10.42   dn01.bigdata
192.168.10.43   dn02.bigdata
```
---

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark
import pandas as pd
import os
from datetime import datetime
from delta import *
from delta.tables import *

In [ ]:
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['JAVA_HOME'] = '/usr/local/jdk8u222-b10'
os.environ['HADOOP_USER_NAME']='hive'
os.environ['PYSPARK_PYTHON'] ='/HDFS01/anaconda3/envs/main/bin/python'
conf = pyspark.SparkConf().setAll([
     ('spark.driver.maxResultSize', '0'),
     ('spark.jars.packages', 'io.delta:delta-core_2.12:2.2.0'),
     ('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension'),
     ('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog'),
     ('spark.sql.warehouse.dir', '/user/hive/warehouse/delta_lake/'),
     ('spark.databricks.delta.replaceWhere.constraintCheck.enabled', 'false'),
     ('spark.databricks.delta.replaceWhere.dataColumns.enabled', 'true'),
     ('spark.databricks.delta.schema.autoMerge.enabled','true'),
     ('spark.databricks.io.cache.enabled','true'),
     ('spark.driver.memory', '2g'),
     ('spark.sql.repl.eagerEval.enabled','true'),
     ('hive.strict.managed.tables','false'),
     ('hive.metastore.uris', 'thrift://nn01.bigdata:9083'),
     ('metastore.client.capability.check','false')
    ])
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("myApp") \
        .config(conf=conf) \
        .enableHiveSupport() \
        .getOrCreate();

## Read CSV and Infer Schema

In [ ]:
df = spark.read.csv("file:///HDFS01/airflow/notebooks/Pasit/PySpark Tutorial/round-1to2-line-lists.csv",
                      header=True, inferSchema=True)

## เขียนลง HDFS Delta Type
* .option("replaceWhere", "start_date >= '2017-01-01' AND end_date <= '2017-01-31'") คือเขียนทับหากตรงเงื่อนไข Where

* .option("maxRecordsPerFile", "10000") คือ จำกัด Rows ในการเขียนลง HDFS สามารถ static ใน Spark Config ได้ตังนี้ spark.sql.files.maxRecordsPerFile=Number_of_rows

* .option("overwriteSchema", "true") คือ สามารถเปลี่ยนประเภทหรือชื่อของคอลัมน์ หรือลบคอลัมน์โดยการเขียนตารางใหม่

* .partitionBy(<your-partition-columns>) คือการสร้าง Partition โดยการใส่ชื่อ Columns

In [ ]:
df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", True) \
	.partitionBy("province") \
    .saveAsTable("delta_lake.covid_summery")

## เปลี่ยนแปลงชื่อ Column โดยใช้วิธีอ่านแล้ว Overwrite กลับไปยัง table

In [ ]:
spark.read.table(...) \
  .withColumnRenamed("dateOfBirth", "birthDate") \
  .write \
  .format("delta") \
  .mode("overwrite") \
  .option("overwriteSchema", "true") \
  .saveAsTable(...)

## อ่าน Delta

In [ ]:
df = spark.read.format("delta").load("/user/hive/warehouse/delta_lake/navy")

## เปิด Transaction เพื่อสามารถ Update, Delete

In [ ]:
dt = DeltaTable.forPath(spark, "/user/hive/warehouse/delta_lake/navy")

## Update Rows

In [ ]:
dt.update(
    "id = '1'",{ "id": "'2909'" }
	)

## Delete Rows

In [ ]:
dt.delete("name = 'Name'")

## สร้าง Delta Table

In [ ]:
DeltaTable.create(spark) \
  .tableName("default.people10m") \
  .addColumn("id", "INT") \
  .addColumn("firstName", "STRING") \
  .addColumn("middleName", "STRING") \
  .addColumn("lastName", "STRING", comment = "surname") \
  .addColumn("gender", "STRING") \
  .addColumn("birthDate", "TIMESTAMP") \
  .addColumn("ssn", "STRING") \
  .addColumn("salary", "INT") \
  .partitionedBy("gender") \
  .execute()

## แสดง Version ของ Metadata

In [ ]:
dt.history() #ถ้าระบุเลขใน () จะเป็นรูปแบบการ Limit

## Restore ไปยัง Version ที่ต้องการ

<b style="color: red;">restore จะใช้งานไม่ได้ใน PrestoDB แต่ถ้า UPDATE INSERT DELETE ปกติในรูปแบบ Append จะยังใช้งานได้</b>

In [ ]:
dt.restoreToVersion(0)

## Optimize Table ปรับโครงสร้างไฟล์ใหม่

* แบบ SQL Statement
* แบบ Spark Build-in

In [ ]:
#1
#spark.sql("OPTIMIZE delta_lake.delta")
#spark.sql("VACUUM delta_lake.delta")

#2
deltaTable = DeltaTable.forName(spark, "delta_lake.delta_stable")
deltaTable.optimize().executeCompaction()

## ใช้ Spark Read Table แบบระบุ Checkpoint version

In [ ]:
df = spark.read \
          .format("delta") \
          .option("versionAsOf", 1) \
          .load("/user/hive/warehouse/delta_lake/navy")
df.show()